# Extract Job Posts from Indeed

Before extracting job posts from [Indeed](https://www.indeed.com/), make sure you have checked their [robots.txt](https://www.indeed.com/robots.txt) file. 

## Create a table in database

In [1]:
import pandas
import configparser
import psycopg2

Read the database connection info from the config.ini

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
db = config['myaws']['db']
user = config['myaws']['user']
pwd = config['myaws']['pwd']

Establish a connection to the databas, and create a cursor.

In [3]:
conn = psycopg2.connect(host = host,
                       user = user,
                        password = pwd,
                        dbname = db
                       )
cur = conn.cursor()

Design the table in SQL

In [4]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp21
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

create the table

In [5]:
cur.execute(table_sql)
conn.commit()

## Request HTML

[urllib.request](https://docs.python.org/3/library/urllib.request.html) makes simple HTTP requests to visit a web page and get the content via the Python standard library.
Here we define the URL to search job pots about Intelligence analyst.

In [6]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=0'

In [7]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))

# Parese HTML
We can use the inspector tool in browsers to analyze webpages and use [beautifulsoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) to extract webpage data.
pip install the beautiful soup if needed. 

In [ ]:
!pip install beautifulsoup4

In [8]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

Use the <code> tag.find_all(‘tag_name’, tage_attr = ‘possible_value’)</code> function to return a list of tags where the attribute equals the possible_value.
<ul>Common attributes include:
<li> id</li>
<li> class<span style ="color:red">_</span></li>
</ul>

<ul>Common functions include:
<li> tag.text: return the visible part of the tag</li>
<li> tag.get(‘attribute’): return the value of the attribute of the tag</li>
</ul>

Since all the job posts are in the <code>div tag class = 'jobsearch-Sprep...'</code>, we need to find that div tag from the body tag.

In [9]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [10]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [11]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

## Save Data to Database
Now we find the div tag contains the job posts. We need to identify the job title, company, ratings, reviews, salary, and summary. 
We can save those records to our table in the database.

In [12]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp21(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

## View the Table

In [13]:
df = pandas.read_sql_query('select * from indeed_gp21', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Undergraduate Internship/Co-op Program - Intel...,Central Intelligence Agency,"Washington, DC",$21.21 - $22.43 an hour,"Engineering, science students, or those in oth..."
1,2,DHS Watch Team Intelligence Analyst,Dynamic Planning & Response,"Arlington, VA","$80,000 - $84,000 a year",Must have a specific level of security clearan...
2,3,Associate Intelligence Analyst,Global Tel*Link,"Raleigh, NC",None,Assist facility staff with investigations by u...
3,4,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Analyst can identify risks in security sys...
4,5,Associate Cyber Intelligence Analyst,LookingGlass Cyber Solutions,"Reston, VA 20191",None,"Past experience in intelligence, technology, c..."
5,6,Undergraduate Internship Program - Resource An...,Central Intelligence Agency,"Washington, DC",$20.75 an hour,Working on teams embedded with mission leaders...
6,7,Political Analyst,Central Intelligence Agency,"Washington, DC","$55,539 - $82,326 a year","The DA helps provide timely, accurate and obje..."
7,8,Intelligence Analyst I - MRS,WorldAware,None,None,Monitors global events through designated tool...
8,9,Intelligence Analyst,General Dynamics Information Technology,None,None,"For more than 50 years, General Dynamics Infor..."
9,10,Intelligence Analyst,BAE Systems,None,None,Having an understanding of physical and techni...


In [14]:
cur.close()
conn.close()